In [1]:
from multiprocessing.managers import State

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langgraph.checkpoint.sqlite import SqliteSaver

In [3]:
class AgentState(TypedDict):
    lnode:str
    scratch:str
    count: Annotated[int, operator.add]

In [6]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {"lnode":"node1", 'count':1,}

def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {"lnode":"node2", 'count':1,}

In [5]:
def should_continue(state:AgentState):
    return state['count'] < 3

In [7]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)
builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", should_continue,
                              {True:"Node1", False:END})
builder.set_entry_point("Node1")

In [8]:
memory = SqliteSaver.from_conn_string(":memory:")
graph = builder.compile(checkpointer=memory)

In [16]:
thread = {"configurable": {"thread_id": str(10)}}
graph.invoke({'scratch':'hi','count':0}, thread)

node1, count:0
node2, count:1
node1, count:2
node2, count:3


{'lnode': 'node2', 'scratch': 'hi', 'count': 4}

In [17]:
graph.get_state(thread)

StateSnapshot(values={'lnode': 'node2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-1689-6a96-8004-675c8abfd1d0'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node2', 'count': 1}}}, created_at='2025-01-31T13:27:34.324085+00:00', parent_config={'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-167f-6e87-8003-1a1cf3d9d3be'}})

In [18]:
for state in graph.get_state_history(thread):
    print(state, "\n")

StateSnapshot(values={'lnode': 'node2', 'scratch': 'hi', 'count': 4}, next=(), config={'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-1689-6a96-8004-675c8abfd1d0'}}, metadata={'source': 'loop', 'step': 4, 'writes': {'Node2': {'lnode': 'node2', 'count': 1}}}, created_at='2025-01-31T13:27:34.324085+00:00', parent_config={'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-167f-6e87-8003-1a1cf3d9d3be'}}) 

StateSnapshot(values={'lnode': 'node1', 'scratch': 'hi', 'count': 3}, next=('Node2',), config={'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-167f-6e87-8003-1a1cf3d9d3be'}}, metadata={'source': 'loop', 'step': 3, 'writes': {'Node1': {'lnode': 'node1', 'count': 1}}}, created_at='2025-01-31T13:27:34.320090+00:00', parent_config={'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-1677-6c6f-8002-e2753d0af3ef'}}) 

StateSnapshot(values={'lnode': 'node2', 'scratch': 'hi', 'count': 2}, next=('Node1',), config={'configurable': {'thread_id': '10', 'thread_t

In [19]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values['count'])

{'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-1689-6a96-8004-675c8abfd1d0'}} 4
{'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-167f-6e87-8003-1a1cf3d9d3be'}} 3
{'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-1677-6c6f-8002-e2753d0af3ef'}} 2
{'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-1667-6924-8001-025cfb6504f3'}} 1
{'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-165a-6703-8000-41f7fb02bb39'}} 0
{'configurable': {'thread_id': '10', 'thread_ts': '1efdfd72-1650-60fa-bfff-bf65bb766dbf'}} 0


In [20]:
states[-3]

{'configurable': {'thread_id': '10',
  'thread_ts': '1efdfd72-1667-6924-8001-025cfb6504f3'}}